In [11]:
import librosa
import numpy as np
import os

# Função 1: Calcular a distância média dos MFCCs
def calculate_mfcc_distance(audio_generated, audio_reference, sr=22050):
    y_gen, _ = librosa.load(audio_generated, sr=sr)
    y_ref, _ = librosa.load(audio_reference, sr=sr)

    mfcc_gen = librosa.feature.mfcc(y=y_gen, sr=sr, n_mfcc=13)
    mfcc_ref = librosa.feature.mfcc(y=y_ref, sr=sr, n_mfcc=13)

    distance = np.mean(np.linalg.norm(mfcc_gen - mfcc_ref, axis=0))
    return distance

# Função 2: Calcular convergência espectral
def spectral_convergence(audio_generated, audio_reference, sr=22050):
    y_gen, _ = librosa.load(audio_generated, sr=sr)
    y_ref, _ = librosa.load(audio_reference, sr=sr)

    S_gen = np.abs(librosa.stft(y_gen))
    S_ref = np.abs(librosa.stft(y_ref))

    convergence = np.linalg.norm(S_gen - S_ref) / np.linalg.norm(S_ref)
    return convergence

# Função 3: Similaridade do histograma de pitch
import numpy as np

def pitch_histogram_similarity(pitch_generated, pitch_reference):
    """
    Calcula a similaridade entre os histogramas de pitch das músicas gerada e de referência.

    pitch_generated: lista com os pitches do áudio gerado.
    pitch_reference: lista com os pitches do áudio de referência.
    Retorna:
        - similaridade entre os histogramas (float entre 0 e 1).
    """
    hist_gen, _ = np.histogram(pitch_generated, bins=128, range=(0, 128))
    hist_ref, _ = np.histogram(pitch_reference, bins=128, range=(0, 128))

    # Normalizar histogramas
    hist_gen = hist_gen / np.sum(hist_gen)
    hist_ref = hist_ref / np.sum(hist_ref)

    # Calcular similaridade
    similarity = np.sum(np.minimum(hist_gen, hist_ref))
    return similarity

# Função 4: Relação Sinal-Ruído (SNR)
def signal_to_noise_ratio(audio_generated, audio_reference, sr=22050):
    y_gen, _ = librosa.load(audio_generated, sr=sr)
    y_ref, _ = librosa.load(audio_reference, sr=sr)

    noise = y_gen - y_ref
    snr = 10 * np.log10(np.sum(y_ref**2) / np.sum(noise**2))
    return snr

# Função principal para calcular todas as métricas
def evaluate_audio(audio_generated, audio_reference, pitch_generated=None, pitch_reference=None, sr=22050):
    results = {}

    # Calcular MFCC Distance
    results['mfcc_distance'] = calculate_mfcc_distance(audio_generated, audio_reference, sr)

    # Calcular Spectral Convergence
    results['spectral_convergence'] = spectral_convergence(audio_generated, audio_reference, sr)

    # Calcular SNR
    results['snr'] = signal_to_noise_ratio(audio_generated, audio_reference, sr)

    # Calcular Similaridade de Histograma de Pitch, se fornecido
    if pitch_generated is not None and pitch_reference is not None:
        results['pitch_histogram_similarity'] = pitch_histogram_similarity(pitch_generated, pitch_reference)
    else:
        results['pitch_histogram_similarity'] = None

    return results

In [ ]:
import mido

def extract_pitches_and_durations(midi_path):
    """
    Extrai pitches e durações de notas de um arquivo MIDI usando mido.

    midi_path: caminho do arquivo MIDI.
    Retorna:
        - pitches: lista de alturas das notas.
        - durations: lista de durações correspondentes às notas.
    """
    pitches = []
    durations = []
    midi = mido.MidiFile(midi_path)
    
    note_on_times = {}
    
    for msg in midi:
        if msg.type == 'note_on' and msg.velocity > 0:
            note_on_times[msg.note] = msg.time  # Marcar o tempo inicial
            pitches.append(msg.note)
        elif msg.type == 'note_off' or (msg.type == 'note_on' and msg.velocity == 0):
            if msg.note in note_on_times:
                durations.append(msg.time - note_on_times[msg.note])  # Calcular duração
                del note_on_times[msg.note]

    return pitches, durations



In [12]:
def evaluate_midi_simplified(midi_generated, midi_reference):
    """
    Avalia dois arquivos MIDI baseando-se em pitches e durações extraídos.

    midi_generated: caminho para o arquivo MIDI gerado.
    midi_reference: caminho para o arquivo MIDI de referência.
    
    Retorna:
        - Dicionário com a métrica de similaridade de pitches.
    """
    # Extrair pitches e durações
    pitch_gen, _ = extract_pitches_and_durations(midi_generated)
    pitch_ref, _ = extract_pitches_and_durations(midi_reference)

    # Calcular similaridade do histograma de pitches
    pitch_similarity = pitch_histogram_similarity(pitch_gen, pitch_ref)

    return {"pitch_histogram_similarity": pitch_similarity}


In [25]:
import librosa
import numpy as np
import mido

# Função 1: Extração de Pitches e Durações
def extract_pitches_and_durations(midi_path):
    pitches = []
    durations = []
    midi = mido.MidiFile(midi_path)

    # Dicionário para armazenar o tempo de início das notas
    note_on_times = {}

    for msg in midi:
        if msg.type == 'note_on' and msg.velocity > 0:
            note_on_times[msg.note] = msg.time  # Tempo inicial da nota
            pitches.append(msg.note)  # Armazena o pitch
        elif msg.type == 'note_off' or (msg.type == 'note_on' and msg.velocity == 0):
            if msg.note in note_on_times:
                duration = msg.time - note_on_times[msg.note]  # Calcula a duração
                durations.append(duration)
                del note_on_times[msg.note]  # Remove a nota processada

    return pitches, durations

# Função 2: Similaridade do Histograma de Pitches
def pitch_histogram_similarity(pitch_generated, pitch_reference):
    hist_gen, _ = np.histogram(pitch_generated, bins=128, range=(0, 128))
    hist_ref, _ = np.histogram(pitch_reference, bins=128, range=(0, 128))

    # Normalizar histogramas
    hist_gen = hist_gen / np.sum(hist_gen)
    hist_ref = hist_ref / np.sum(hist_ref)

    # Calcular similaridade
    similarity = np.sum(np.minimum(hist_gen, hist_ref))
    return similarity

# Função 3: Distância de MFCC
def calculate_mfcc_distance(audio_generated, audio_reference, sr=22050):
    y_gen, _ = librosa.load(audio_generated, sr=sr)
    y_ref, _ = librosa.load(audio_reference, sr=sr)

    # Ajustar os sinais para o mesmo comprimento
    min_length = min(len(y_gen), len(y_ref))
    y_gen = librosa.util.fix_length(y_gen, size=min_length)
    y_ref = librosa.util.fix_length(y_ref, size=min_length)

    mfcc_gen = librosa.feature.mfcc(y=y_gen, sr=sr, n_mfcc=13)
    mfcc_ref = librosa.feature.mfcc(y=y_ref, sr=sr, n_mfcc=13)

    distance = np.mean(np.linalg.norm(mfcc_gen - mfcc_ref, axis=0))
    return distance

# Função 4: Convergência Espectral
def spectral_convergence(audio_generated, audio_reference, sr=22050):
    y_gen, _ = librosa.load(audio_generated, sr=sr)
    y_ref, _ = librosa.load(audio_reference, sr=sr)

    # Ajustar os sinais para o mesmo comprimento
    min_length = min(len(y_gen), len(y_ref))
    y_gen = librosa.util.fix_length(y_gen, size=min_length)
    y_ref = librosa.util.fix_length(y_ref, size=min_length)

    S_gen = np.abs(librosa.stft(y_gen))
    S_ref = np.abs(librosa.stft(y_ref))

    convergence = np.linalg.norm(S_gen - S_ref) / np.linalg.norm(S_ref)
    return convergence

# Função 5: Relação Sinal-Ruído (SNR)
def signal_to_noise_ratio(audio_generated, audio_reference, sr=22050):
    y_gen, _ = librosa.load(audio_generated, sr=sr)
    y_ref, _ = librosa.load(audio_reference, sr=sr)

    # Ajustar os sinais para o mesmo comprimento
    min_length = min(len(y_gen), len(y_ref))
    y_gen = librosa.util.fix_length(y_gen, size=min_length)
    y_ref = librosa.util.fix_length(y_ref, size=min_length)

    noise = y_gen - y_ref
    snr = 10 * np.log10(np.sum(y_ref**2) / np.sum(noise**2))
    return snr

# Função Principal para Avaliação
def evaluate_midi_and_audio(midi_generated, midi_reference, audio_generated, audio_reference, sr=22050):
    """
    Avalia arquivos MIDI e áudios correspondentes com métricas simbólicas e baseadas em waveform.
    
    midi_generated: caminho para o MIDI gerado.
    midi_reference: caminho para o MIDI de referência.
    audio_generated: caminho para o áudio gerado (renderizado do MIDI gerado).
    audio_reference: caminho para o áudio de referência (renderizado do MIDI de referência).
    sr: taxa de amostragem desejada.
    
    Retorna:
        - Dicionário com todas as métricas calculadas.
    """
    results = {}

    # Métricas Simbólicas
    pitch_gen, _ = extract_pitches_and_durations(midi_generated)
    pitch_ref, _ = extract_pitches_and_durations(midi_reference)
    results['pitch_histogram_similarity'] = pitch_histogram_similarity(pitch_gen, pitch_ref)

    # Métricas Baseadas em Áudio
    results['mfcc_distance'] = calculate_mfcc_distance(audio_generated, audio_reference, sr)
    results['spectral_convergence'] = spectral_convergence(audio_generated, audio_reference, sr)
    results['signal_to_noise'] = signal_to_noise_ratio(audio_generated, audio_reference, sr)

    return results


In [26]:

# Caminhos para os arquivos MIDI
midi_generated = r"C:\Users\not\Documents\Estudos\Unicamp - Mestrado\MuseMorphose\generations\id23_bar7_sample01_poly-1_rhym+1.mid"
midi_reference = r"C:\Users\not\Documents\Estudos\Unicamp - Mestrado\MuseMorphose\generations\id23_bar7_orig.mid"

# Caminhos para os áudios correspondentes
audio_generated = r"C:\Users\not\Documents\Estudos\Unicamp - Mestrado\MuseMorphose\generations\id23_bar7_sample01_poly-1_rhym+1.wav"  # Gerado a partir do MIDI
audio_reference = r"C:\Users\not\Documents\Estudos\Unicamp - Mestrado\MuseMorphose\generations\id23_bar7_orig.wav"  # Gerado a partir do MIDI

# Avaliar os arquivos
metrics = evaluate_midi_and_audio(
    midi_generated=midi_generated,
    midi_reference=midi_reference,
    audio_generated=audio_generated,
    audio_reference=audio_reference
)

# Exibir os resultados
print("Resultados da Avaliação:")
for metric, value in metrics.items():
    print(f"{metric}: {value}")


Resultados da Avaliação:
pitch_histogram_similarity: 0.6907216494845361
mfcc_distance: 63.088687896728516
spectral_convergence: 0.9994791150093079
signal_to_noise: -2.211090564727783


#### mfcc_distance: MFCC (Mel Frequency Cepstral Coefficients) 

Representa as características espectrais do áudio, como a tonalidade e o timbre.
A distância MFCC mede o quão diferentes são as características espectrais entre dois áudios.

Interpretação:
1. Valores Baixos: Os áudios são mais similares em termos de espectro e timbre.
2. Valores Altos: Os áudios têm características espectrais muito diferentes.

Valores típicos:
1. <10: Áudios muito similares.
2. 10–30: Similaridade moderada.
3. \>30: Áudios significativamente diferentes.

#### spectral_convergence: Spectral Convergence

Mede a diferença entre os espectrogramas (representações das frequências ao longo do tempo) dos dois áudios.
Calcula a relação entre a norma da diferença e a norma do espectrograma de referência.

Interpretação:
1. Valores Baixos: Alta semelhança espectral (ex.: mesmo padrão de frequência ao longo do tempo).
2. Valores Altos: Grande discrepância nas frequências e sua evolução.

Valores típicos:
1. \<0.2: Semelhança espectral alta.
2. 0.2–0.5: Moderada semelhança espectral.
3. \>0.5: Diferenças significativas.

#### snr: Signal-to-Noise Ratio (SNR)

Mede a relação entre o sinal útil (áudio de referência) e o ruído (diferença entre os dois áudios).
Um SNR alto indica que o áudio gerado é muito semelhante ao de referência.

Interpretação:
1. Valores Altos (dB): Indicam que o áudio gerado é semelhante ao de referência e tem menos "ruído".
2. Valores Baixos (dB): Indicam que o áudio gerado contém muitos elementos indesejados.

Valores típicos:
1. \>20 dB: Muito semelhante.
2. 10–20 dB: Moderada semelhança.
3. \<10 dB: Diferenças significativas ou ruído perceptível.

#### pitch_histogram_similarity: Pitch Histogram Similarity

Compara a distribuição das alturas das notas (pitches) entre os dois áudios, útil para músicas simbólicas (ex.: MIDI).
Mede a semelhança entre os histogramas de pitch.

Interpretação:
1. Valores Próximos de 1: Alta correspondência entre as alturas das notas.
2. Valores Próximos de 0: Grande discrepância nas alturas das notas.

Valores típicos:
1. \>0.8: Altas similaridades.
2. 0.5–0.8: Moderada correspondência.
3. \<0.5: Diferente em termos de notas tocadas.